In [1]:
import pandas as pd
import numpy as np

Loading input into numpy ndarray

In [2]:
def clean_cell(cell: str):
    if cell == "R\n" or "M\n":
        return cell.strip("\n")
    return float(cell)

In [3]:
file = "data/sonar.all-data"
f = open(file)
M = []
for line in f:
    M.append([clean_cell(x) for x in line.split(',')])
M = np.array([np.array(i) for i in M])

Splitting matrix into features and classes

In [4]:
x = M[:, :60].astype(float)
y = M[:, 60]

Normalising features

In [5]:
def normalise_2darray(d2array):
    output_array = []
    for array in d2array:
        output_array.append((array - np.mean(array)) / np.std(array))
    return np.asarray(output_array)

x_norm = normalise_2darray(x)

Encoding classes and displaying counts

In [6]:
def encode_classes(target):
    encoded_target = []
    encoding_dict = {'R': 0, 'M': 1}
    
    for x in target:
        encoded_target.append(encoding_dict.get(x))
            
    return np.asarray(encoded_target), encoding_dict

y_enc, encoding = encode_classes(y)
np.array(np.unique(y_enc, return_counts=True)).T

array([[  0,  97],
       [  1, 111]], dtype=int64)

Removing features with low impact on target class

In [7]:
corr_arr = []
for idx, row in enumerate(x_norm):
    arr = np.asarray(list(row) + [y_enc[idx]])
    corr_arr.append(arr)
corr_arr = np.asarray(corr_arr)
corr_map = np.corrcoef(corr_arr, rowvar=False).round(2)
corr_map = corr_map[:, 60]  # keep only final column of the heatmap | correlation to target class
corr_map = corr_map.reshape((61, 1))

to_drop = []
for idx, value in enumerate(corr_map):
    if value > -0.1 and value < 0.1:
        to_drop.append(idx)
to_drop

x_norm = np.delete(x_norm, to_drop, axis=1)
x_norm.shape

(208, 37)

Splitting into train, val and test sets

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test  = train_test_split(x_norm, y_enc, test_size=0.1)
x_train, x_val, y_train, y_val =  train_test_split(x_train, y_train, test_size=0.3)

In [9]:
print(x_train.shape, x_val.shape, x_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(130, 37) (57, 37) (21, 37)
(130,) (57,) (21,)


In [10]:
np.array(np.unique(y_train, return_counts=True)).T

array([[ 0, 64],
       [ 1, 66]], dtype=int64)

In [11]:
np.array(np.unique(y_val, return_counts=True)).T

array([[ 0, 22],
       [ 1, 35]], dtype=int64)

In [12]:
np.array(np.unique(y_test, return_counts=True)).T

array([[ 0, 11],
       [ 1, 10]], dtype=int64)

Creating model in Keras (tensorflow)

In [13]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.activations import sigmoid, relu, hard_sigmoid, tanh, softmax

input = Input(shape=(x_norm.shape[1], ), name="input")
dense1 = Dense(128, activation=lambda x: relu(x, alpha=0.3))(input,)
dense2 = Dense(128, activation=lambda x: relu(x, alpha=0.3))(dense1)
dense3 = Dense(128, activation=lambda x: relu(x, alpha=0.3))(dense2)
dense4 = Dense(64, activation=lambda x: relu(x, alpha=0.3))(dense3)
dense5 = Dense(64, activation=lambda x: relu(x, alpha=0.3))(dense4)
dense6 = Dense(64, activation=lambda x: relu(x, alpha=0.3))(dense5)
dense_out = Dense(1, activation=sigmoid, name="dense_out")(dense6)
model = Model(input, dense_out, name='test_model')

c:\Users\vikio\anaconda3\envs\nsiete_test\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (None)/charset_normalizer (3.0.1) doesn't match a supported version!
  warnings.warn(


The network consists of dense layers using a leaky relu activation function. The alpha is set to 0.3 because it is the default value for the LeakyReLu Keras layer and also more or less consistent with solution and literature I found online. I am using leaky relu instead of standard relu to avoid dead gradients.

In [14]:
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.metrics import BinaryAccuracy, AUC

optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99)
model.compile(loss=BinaryCrossentropy(from_logits=False), optimizer=optimizer, metrics=[BinaryAccuracy(), AUC()])

We are using the Adam optimizer with default arguments. The model is evaluated using the BinaryAccuracy and AUC metrics

In [15]:
val_data = (x_val, y_val)
model.fit(x=x_train, y=y_train, validation_data=val_data, batch_size=5, epochs=25, verbose=1)

Epoch 1/25
26/26 [==============================] - 1s 9ms/step - loss: 0.6510 - binary_accuracy: 0.6000 - auc: 0.6847 - val_loss: 0.7322 - val_binary_accuracy: 0.4386 - val_auc: 0.7435
Epoch 2/25
26/26 [==============================] - 0s 3ms/step - loss: 0.6305 - binary_accuracy: 0.6615 - auc: 0.7041 - val_loss: 0.6644 - val_binary_accuracy: 0.5614 - val_auc: 0.7409
Epoch 3/25
26/26 [==============================] - 0s 3ms/step - loss: 0.4479 - binary_accuracy: 0.7615 - auc: 0.8767 - val_loss: 0.5532 - val_binary_accuracy: 0.7193 - val_auc: 0.7974
Epoch 4/25
26/26 [==============================] - 0s 3ms/step - loss: 0.3936 - binary_accuracy: 0.8308 - auc: 0.9029 - val_loss: 0.5348 - val_binary_accuracy: 0.7193 - val_auc: 0.7987
Epoch 5/25
26/26 [==============================] - 0s 3ms/step - loss: 0.3604 - binary_accuracy: 0.8692 - auc: 0.9182 - val_loss: 0.6704 - val_binary_accuracy: 0.5965 - val_auc: 0.8377
Epoch 6/25
26/26 [==============================] - 0s 3ms/step - loss

In [16]:
model.evaluate(x=x_test, y=y_test)

1/1 [==============================] - 0s 18ms/step - loss: 0.5543 - binary_accuracy: 0.8095 - auc: 0.9545


[0.5542677640914917, 0.8095238208770752, 0.9545454382896423]

In [17]:
model.summary()

Model: "test_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 37)]              0         
                                                                 
 dense (Dense)               (None, 128)               4864      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 64)                4